# Neural Network

In [18]:
import numpy as np
import pandas as pd
import tensorflow as tf 
import math
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

In [2]:
filePath = "bank-additional\\bank-additional\\bank-additional-full.csv"
data = pd.read_csv(filePath, sep=";")

## Preprocessing

### Encoding categorical variables
Transforming non-numeric labels into numeric labels.

In [3]:
categorical_label_encoder = LabelEncoder()

for variable in data.select_dtypes(include=["object"]).columns:
    data[variable] = categorical_label_encoder.fit_transform(data[variable])

### Seperating features and target
Seperating/Dropping the target variable (AKA the output variable) from the features (input variables). This is done because the model the target variable itself isn't used to train the model, instead the features are to train the model in order to predict the target variable; therefore, to prevent the model from learning incorrect patterns and because the model is evaluated on the target variable, it is necessary to drop the target variable.

In [4]:
features = data.drop("y", axis=1)
target = data["y"]

### Standardization
The aim of Standardization is to increase data quality through eliminating errors, inconsistencies and redundant data. Through having a higher quality dataset, the model will be able to learn patterns from more verbose data leading to better predictions.

In [5]:
standard_scaler = StandardScaler()
scaled_features = standard_scaler.fit_transform(features)

### Train Test Split
A dataset is split into two parts, the training dataset and the testing dataset. This done so a model can be tested/evaluated after training it, without testing the model, it won't be possbile to find out whether a it is underfitted or overfitted (Bias-Variance Tradeoff).

In [6]:
X_train, X_test, y_train, y_test = train_test_split(scaled_features, target, train_size=0.8, test_size=0.2, random_state=42)

## Creating the Neural Network Model

This model will be using a Multilayer Perceptron with an input layer, a hidden layer and an output layer. 

In [7]:
number_of_features = features.shape[1]
number_of_features

20

### Deciding the Number of Neurons

#### Deciding the Number of Neurons in the Input Layer and the Output Layer
Since there are 20 inputs, there will be 20 neurons in the input layer. Since an instance can only be classified into one of two types, this model is using **Binary Classification**, therefore it will be using one neuron for the output.

#### Deciding the Number of Neurons in the Hidden Layer/Layers
However, deciding the number of Neurons in the hidden layers, or in this case, the hidden layer is not as straightforward as the other layers. The complexity of the dataset (dimensionality, sample size, noise, distribution, etc) and whether the model is underfit or overfit has an effect on how many neurons should be in the hidden layer.

According to [The Number of Hidden Layers](https://www.heatonresearch.com/2017/06/01/hidden-layers.html), there are a few rules of thumb for determining the number of neurons in a hidden layer (Additionally, this article discusses about Deep Learning and why multiple hidden layers are used despite the Universal Approximation Theorem proving that a single hidden layer neural network can learn anything). 

1. The average of the sum of the number of Input and Output Neurons.
2. Sum of 2/3rds of the number of Input Neurons and the number of Output Neurons.
3. Less than two times the number of Input Neurons

The first heuristic tends to give a smaller number of hidden neurons, since there are benefits to a smaller networks such as a faster training time and being less prone to overfitting, therefore we will be deciding hte number of hidden neurons by getting the average of the sum of of the number of Input and Output Neurons.

In [8]:
number_of_input_neurons = number_of_features
number_of_output_neurons = 1
number_of_hidden_neurons = math.ceil((number_of_input_neurons + number_of_output_neurons) / 2)
number_of_hidden_neurons

11

### Deciding the Activation Function
For the activation function, ReLU will be due to the fact that it is simple and efficient, it is also the standard activation function that is used for classification tasks, with other functions such as sigmoid or Tanh being used for situations where ReLU isn't optimal.

### Coding the Model

In [9]:
input_layer = Dense(number_of_input_neurons, input_shape=(number_of_features,))
hidden_layer = Dense(number_of_hidden_neurons, activation="relu")
output_layer = Dense(number_of_output_neurons)

neural_network_model = Sequential([input_layer, hidden_layer, output_layer])

c:\Users\Mail2\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


### Compiling

#### Optimizer
An Optimizer modifies the weights during training to reduce the difference between predicted values and actual values, also known as **'Loss'**. There are many optimizers such as Gradient Descent, Stochastic Gradient Descent, RMSprop, Adam, etc. Optimizers are divided into two groups according to how the **learning rate** is modified. If you have to manually tune the learning rate, it is a **Gradient Descent Algorithm** and if it can automatically adapt, it is a **Adaptive Algorithm**. This model will be using an Adaptive Optimizer called **'Adam'** due to its ease of use.

#### Loss Function
A Loss Function is used to measure how well a model can predict values, this is done by taking the difference between a predicted value and its corresponding actual value, the difference between these two are called **'Loss'**. This model will be using **Binary Cross Entropy** also known as **Logarithmic Loss** or **Log Loss**, this loss function is being used because it makes the model more likely to have output values close to 0 or 1 and because it is standard for binary classification.

The aforementioned Optimizer uses the loss from a loss function to modify the weights during training.

In [10]:
neural_network_model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

## Training The Neural Network Model
### Batch Size
The Batch Size is a hyperparameter that defines the **number of samples/datapoints** to work through before updating the model's internal parameters as weights, biases, etc.

Batches are used because updating the parameters on the entire training dataset is computationally expensive (unless the dataset is small) and because it leads to a shorter training time. However having a large batch size might mean that the model is less accurate and prone to overfitting.

### Number of Epochs
The Number of Epochs is a hyperparameter that defines the **number of training iterations** for the model.

As the model goes through each epoch, its parameters are updated and patterns in the dataset are learned. It is important to have a balanced amount of epochs as too few leads to underfitting and the inability of the model to extract patterns from the dataset and on the other hand, having too many epochs leads to the model being overfit, which leads to poor predictions on new data.


In [11]:
trained_neural_network = neural_network_model.fit(X_train, y_train, batch_size=32, epochs=32, validation_split=0.2, verbose=1)

Epoch 1/32
824/824 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.8245 - loss: 1.8723 - val_accuracy: 0.8657 - val_loss: 1.4038
Epoch 2/32
824/824 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8739 - loss: 1.2037 - val_accuracy: 0.8829 - val_loss: 0.7788
Epoch 3/32
824/824 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8851 - loss: 0.7772 - val_accuracy: 0.8876 - val_loss: 0.8058
Epoch 4/32
824/824 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8930 - loss: 0.6879 - val_accuracy: 0.8958 - val_loss: 0.6110
Epoch 5/32
824/824 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8940 - loss: 0.5477 - val_accuracy: 0.8935 - val_loss: 0.4951
Epoch 6/32
824/824 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8920 - loss: 0.4824 - val_accuracy: 0.8888 - val_loss: 0.4605
Epoch 7/32
824/824 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8950 - loss: 0.4564 - val_accuracy: 0.8689 - val_loss: 0.5587
Epoch 8/32
824/824 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8788 - loss: 0.5115 - val_accuracy: 0.

## Testing

In [25]:
predictions = (neural_network_model.predict(X_test) > 0.5).astype("int32")
print(accuracy_score(y_test, predictions))
print(classification_report(y_test, predictions))

258/258 ━━━━━━━━━━━━━━━━━━━━ 0s 873us/step
0.9068948773974266
              precision    recall  f1-score   support

           0       0.92      0.98      0.95      7303
           1       0.69      0.33      0.44       935

    accuracy                           0.91      8238
   macro avg       0.80      0.65      0.70      8238
weighted avg       0.89      0.91      0.89      8238

